## Granger Causality and Inverse Probability Weighting
**Learning Objectives**
* Protocol Buffers in Python
* Imputing Missing Data using semi-supervised learning with forecast model - just as with stock markets, unexpected events may occur. Although they may not fully be captured in the intended ARIMA model's noise terms, it is done for imputation so protocol buffers types within messages may be used.
* Granger Causality Implementation
* Making causal claims - difference in expected outcomes - using inverse probability weighting

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from pandas.plotting import autocorrelation_plot

In [2]:
attacksData = pd.read_csv('./malicious_dataset.csv') 
normalData = pd.read_csv('./normal_dataset.csv')


In [3]:
normalData['label'] = 'normal' # adds label column to be compatible with attacksData when concatenating

In [4]:
normalAndAttack = pd.concat([normalData,attacksData])

In [5]:
normalAndAttack = normalAndAttack.rename(columns={s: (s.replace(".","_")) for s in list(normalAndAttack.columns)})

In [204]:
# noting less meaningful features: ip_flags_mf, ip_tos, ip_flags_rb, ip_frag_offset - some were all NaN or 
# mostly NaN with only one other possibility.
# only 4.0.
# ip_version
# ip_hdr_len
# 

In [6]:
normalAndAttack = normalAndAttack.drop(columns=['ip_flags_mf', 'ip_tos', 
                                                'ip_flags_rb', 'ip_frag_offset',
                                                "ip_version","ip_hdr_len",
                                               "frame_info_encap_type"])

In [172]:
# hmms consider observations over time steps, but they look into the probability of 
# hidden tags given the observations. Here, we have a time-series. 
# Here, neural networks are a possibility, but it's a black box approach and convergence 
# is towards a local optima. Convergence may also be slow. One can do a naive mean, a naive equality
# to the previous value or seasonal naive model for time-series data.

In [173]:
# ttl - time to live

In [206]:
months = dict()
months["Jan"] = 1
months["Feb"] = 2
months["Mar"] = 3
months["Apr"] = 4
months["May"] = 5
months["Jun"] = 6
months["Jul"] = 7
months["Aug"] = 8
months["Sep"] = 9
months["Oct"] = 10
months["Nov"] = 11
months["Dec"] = 12

In [175]:
# MSS_VAL is the maximum segment size under the options 
# field of TCP header that informs the largest 
# amount of data in bytes that a machine can receive in a single TCP segment

In [176]:
# getting categorical attributes
# ip_flags
# ip_flags_df
# ip_proto
# tcp_flags_fin
# tcp_flags_syn
# tcp_flags_reset
# tcp_flags_push
# tcp_flags_ack
# tcp_flags_urg
# tcp_flags_cwr

In [207]:
def majorityVoteVal(col):
    counts = dict()
    maxCount, maxVal = 0, None
    for val in col:
        if counts.get(val) == None:
            if 1 >= maxCount:
                maxCount = 1
                maxVal = val
            counts[val] = 1
        else:
            if counts[val]+1 >= maxCount:
                maxCount = counts[val]+1
                maxVal = val
            counts[val] += 1
    return maxVal

In [208]:
# remove nas in these: tcp_srcport, tcp_dstport

In [209]:
valueNaNs = dict()
for col in ["ip_flags", "ip_flags_df", "ip_proto", 
            "tcp_flags_fin", "tcp_flags_syn", "tcp_flags_reset",  
            "tcp_flags_push", "tcp_flags_ack", "tcp_flags_urg",
            "tcp_flags_cwr","tcp_urgent_pointer"]:
    val = majorityVoteVal(list(normalAndAttack[col].dropna()))
    valueNaNs[col] = val
newNormalAndAttack = normalAndAttack.fillna(value=valueNaNs)
#newNormalAndAttack

In [210]:
newNormalAndAttack2 = newNormalAndAttack.dropna(subset=['ip_src', 'ip_dst','tcp_srcport','tcp_dstport'])

In [211]:
newNormalAndAttack2 = newNormalAndAttack2.drop(columns=['ip_proto','eth_type'])

In [212]:
earliest=datetime.datetime.strptime("2019-11-21 02:00:00.313671", '%Y-%m-%d %H:%M:%S.%f')
s = "Nov 2, 2020 16:20:45.136949"
def formatTime(s):
    items = s.split(" ")
    modified_items = list(filter(lambda x: x != '', items))
    day = modified_items[1][:len(modified_items[1])-1]
    if len(modified_items[1]) == 2:
        day = "0" + day
    month = str(months[modified_items[0]])
    if len(month) == 1:
        month = "0" + month
    hours = modified_items[3]
    newdate = modified_items[2]+"-"+ month + "-" + day + " "+hours[:len(hours)-3]
    dateTimeDate =  datetime.datetime.strptime(newdate, '%Y-%m-%d %H:%M:%S.%f')
    def timestamp_microsecond(currDateTime):
        diff = currDateTime - earliest
        return (diff.days * 86400 + diff.seconds) * 10**6 + diff.microseconds
    return timestamp_microsecond(dateTimeDate)
formatTime(s)

30032444822329

In [213]:
newNormalAndAttack2 = newNormalAndAttack2.assign(frame_info_time = newNormalAndAttack2["frame_info_time"].map(formatTime))


In [216]:
newNormalAndAttack2.columns

Index(['frame_info_time', 'frame_info_time_epoch', 'frame_info_number',
       'frame_info_len', 'frame_info_cap_len', 'ip_id', 'ip_flags',
       'ip_flags_df', 'ip_ttl', 'ip_checksum', 'ip_src', 'ip_dst', 'ip_len',
       'ip_dsfield', 'tcp_srcport', 'tcp_dstport', 'tcp_seq', 'tcp_ack',
       'tcp_len', 'tcp_hdr_len', 'tcp_flags', 'tcp_flags_fin', 'tcp_flags_syn',
       'tcp_flags_reset', 'tcp_flags_push', 'tcp_flags_ack', 'tcp_flags_urg',
       'tcp_flags_cwr', 'tcp_window_size', 'tcp_checksum',
       'tcp_urgent_pointer', 'tcp_options_mss_val', 'label'],
      dtype='object')

In [189]:
import datetime

In [157]:
import time 

In [192]:
t = datetime.datetime.strptime("2020-09-02 16:20:45.136949",'%Y-%m-%d %H:%M:%S.%f')
p=datetime.datetime.strptime("2019-11-21 02:00:00.313671", '%Y-%m-%d %H:%M:%S.%f')
def timestamp_microsecond(utc):
    diff = utc - p
    assert diff.resolution == datetime.timedelta(microseconds=1)
    return (diff.days * 86400 + diff.seconds) * 10**6 + diff.microseconds
timestamp_microsecond(t)

24762044823278

In [196]:
#print(len(newNormalAndAttack2["ip_flags"]))
#len(newNormalAndAttack2["ip_flags"].dropna())
#newNormalAndAttack2["tcp_urgent_pointer"].unique() 

In [228]:
labelTypes = list(newNormalAndAttack2["label"].unique())
def getNumericLabels(label):
    return labelTypes.index(label)+1

In [235]:
print(labelTypes)

['normal', 'nmap_null', 'nmap_connect', 'zmap', 'nmap_window', 'masscan', 'hping_syn', 'unicorn_null', 'unicorn_syn', 'nmap_xmas', 'nmap_syn', 'unicorn_conn', 'unicorn_xmas', 'nmap_ack', 'hping_fin', 'nmap_maimon', 'hping_null', 'hping_xmas', 'hping_ack', 'nmap_fin', 'unicorn_fxmas']


In [230]:
newNormalAndAttack2 = newNormalAndAttack2.assign(label = newNormalAndAttack2["label"].map(getNumericLabels))



In [241]:
newNormalAndAttack2['tcp_flags_reset'].unique()

array([0., 1.])

In [234]:
_ = newNormalAndAttack2.to_csv("portScan_data.csv")

## Network Visualization

In [52]:
#!pip3 install pyvis

In [218]:
from pyvis.network import Network

In [219]:
import networkx as nx

In [220]:
#newNormalAndAttack.columns

In [221]:
df = newNormalAndAttack2.sample(100)
G = nx.from_pandas_edgelist(df, source='ip_src', 
                            target='ip_dst',edge_attr=True,
                            create_using=nx.DiGraph())
H = G.to_undirected()

In [140]:
#nx.draw_circular(G,with_labels = True)

In [222]:
g = Network(height=1600,width=1600,notebook=True)
g.toggle_hide_edges_on_drag(True)
g.barnes_hut()
# from_nx only accepts undirected graphs
g.from_nx(H)
g.show("g.html")

In [185]:
#(set(newNormalAndAttack["ip_src"])).intersection(set(newNormalAndAttack["ip_dst"]))

## Handling Missing Values in Time Series
* As with any missing values, one can (1) impute the data or (2) remove the records. 
* Imputation occurs in RStudio - Kalman Filter

In [1]:
# added again for convenience
import numpy as np
import pandas as pd

In [2]:
updatedPortScanDf = pd.read_csv("newPortData.csv")

In [3]:
#updatedPortScanDf.head()

In [4]:
updatedPortScanDf = updatedPortScanDf.drop(columns=['X','Unnamed: 0'])

In [5]:
#updatedPortScanDf.columns

In [6]:
import portData_pb2

In [18]:
updatedPortScanDf.head()

,frame_info_time,frame_info_time_epoch,frame_info_number,frame_info_len,frame_info_cap_len,ip_id,ip_flags,ip_flags_df,ip_ttl,ip_checksum,...,tcp_flags_reset,tcp_flags_push,tcp_flags_ack,tcp_flags_urg,tcp_flags_cwr,tcp_window_size,tcp_checksum,tcp_urgent_pointer,tcp_options_mss_val,label
0,0.0,1.574312e+09,7,54,54,46834,0,0,247,12832,...,0,0,0,0,0,1024,61355,0,17055.139055,1
1,2481.0,1.574312e+09,12,551,66,3793,16384,1,56,13960,...,0,1,1,0,0,252,60933,0,16923.056473,1
2,2556.0,1.574312e+09,14,94,94,0,16384,1,59,12472,...,0,0,1,0,0,4677,18651,0,16738.109830,1
3,3197.0,1.574312e+09,15,68,66,8559,16384,1,55,64767,...,0,1,1,0,0,115,7590,0,16507.535890,1
4,3273.0,1.574312e+09,16,54,54,54321,0,0,244,40274,...,0,0,0,0,0,65535,18914,0,16237.596014,1


In [9]:
def addFrameExamples(dataframe, filename):
    # some borrowed from https://developers.google.com/protocol-buffers/docs/pythontutorial
    # read the existing 
    newPortData = portData_pb2.portDataFrame()
    try:
        f = open(filename, "rb")
        newPortData.ParseFromString(f.read())
        f.close()
    except IOError:
        # then no file available
        print(filename + ": Could not open file.  Creating a new one.")
    cols = list(dataframe.columns)
    for index, row in dataframe.iterrows():
        example = newPortData.examples.add()
        for i in range(len(cols)):
            exec("example.{} = row[{}]".format(cols[i], i))
        # 42 being the fixed number of columns - attributes
        
    f = open(filename, "wb")
    f.write(newPortData.SerializeToString())
    f.close()

addFrameExamples(updatedPortScanDf,"portDataFile")


portDataFile: Could not open file.  Creating a new one.


In [37]:
#!pip3 install read-protobuf
from read_protobuf import read_protobuf

In [38]:

portDataFrame = portData_pb2.portDataFrame()
protoDf = read_protobuf('portDataFile', portDataFrame)


In [46]:
protoDf = protoDf.fillna(0)

In [47]:
protoDf["tcp_urgent_pointer"].unique()

array([0.0000e+00, 1.0000e+00, 1.0000e+01, 4.6335e+04, 4.0571e+04,
       2.5600e+02])

## Protocol Buffers
A Protocol Buffer is one among many data serialization methods such as CSV, JSON, or XML that 
has advantages in datatype support defined in the .proto file and standardization maintained
by Google, but is not intended to be human-readable. The serialization method that 
results in the most human-readable data is TOML, but it's slower than other serialization
methods. There is also less language support with TOML given it was released initially in 2013 
and is relatively inchoate.

## Notions of Causality

The earliest concept of causality in time-series data was granger causality that suggested the difference
between predicting stochastic process $Y$ compared to $X$ given all the information of the "universe". 
Assuming that $X,Y$ are stationary stochastic processes.

If removing $X$ reduces the predictive power regarding $Y$, $X$ contains unique information regarding $Y$ so 
$X$ Granger-causes $Y$.

$U_i = (U_{i_1},...,U_{i_{\infty}})$ contains all the information until time $i$. $\sigma^2 (Y_i|U_i)$
is the variance of predicting $Y_i$ using $U_i$ at time $i$. $\sigma^2 (Y_i|U_i \ X_i)$ excludes $X_i$
when predicting $Y_i$. 

If $\sigma^2 (Y_i|U_i) < \sigma^2 (Y_i|U_i \ X_i)$, then $X$ granger-causes $Y$.

**A feedback** occurs between $X,Y$ when $X$ granger-causes $Y$ and $Y$ granger-causes $X$.

**Practically, we have access to a limited set of observed time series $X$, so we observe $X$ 
granger-causes $Y$ w.r.t $X$**

**Instantaneous causality** occurs between 2 stochastic processes if at time $i$, adding $X_i$
helps improve the predicted value $Y_i$. 

If $\sigma^2(Y_i|U_i \cup \{X_i\}) < \sigma^2(Y_i|U_i)$, then there is instantaneous causality between X and Y.

## Inverse Probability Weighting (IPW)

IPW compensates for underrepresented and oversampled groups by weighting individuals from a particular group by the inverse of the probability of being in that group. Thereby, those in minority groups will be weighed more 
heavily than those in oversampled groups. 

The notion of a pseudo-population occurs with 

The causal estimand is **identifiable** if we have exchangeability  

One can look to root cause detection in anomalous time series. This comes from anomalous behavior
of one of the continuous variables. Another approach involves directly identifying the causal
structures at play with graph search.
**SGS/PC Algorithms**

In [25]:
updatedNumericPortScanDf = updatedPortScanDf.drop(columns=["ip_src","ip_dst"])

In [33]:
# here we use an "adjacency list" representation as a mapping of a feature to every other feature.
# (1) This is the initialization of the fully, densely connected graph
connectedGraph = dict()
numericCols = updatedNumericPortScanDf.columns
colsSet = set(numericCols)
for col in numericCols:
    connectedGraph[col] = colsSet.difference(set([col]))

In [ ]:
# (2) edge elimination occurs when edges are conditionally independent - 
# test for conditional independence can be under PC Algorithm that 
# assumes 
# (3) identifying unshielded colliders - for pairs of variables connected
# through a third variable, test conditional independence on third variable.
# 

In [12]:
from statsmodels.tsa.stattools import grangercausalitytests

### Granger Causality F-test
**With Granger Causality, we can conduct the F-test** by comparing a full model against reduced model.
If the null hypothesis is rejected, we say some stochastic process $X$ granger-causes $Y$. This 
provides insight into how well the previous values of 
one time series can predict the other.

**When doing these analyses, we assuming the causes precede the effect and the cause has
unique information about future values of effect**

In [63]:
grangercausalitytests(updatedPortScanDf[["tcp_options_mss_val","ip_ttl"]], [10])


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.5328  , p=0.1205  , df_denom=284670, df_num=10
ssr based chi2 test:   chi2=15.3296 , p=0.1205  , df=10
likelihood ratio test: chi2=15.3292 , p=0.1205  , df=10
parameter F test:         F=1.5328  , p=0.1205  , df_denom=284670, df_num=10


{10: ({'ssr_ftest': (1.5328469320571256, 0.12054409934638233, 284670.0, 10),
   'ssr_chi2test': (15.329600096050696, 0.12049929983907455, 10),
   'lrtest': (15.329187377355993, 0.12051322041755312, 10),
   'params_ftest': (1.532846930777425, 0.12054409977629976, 284670.0, 10.0)},
   array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
           0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [9]:
#!pip3 install fcit
# we determine if P(y|x, z) = P(y|z)

In [10]:
from fcit import fcit
pvalLenMSS = fcit.test(np.transpose(np.matrix(updatedPortScanDf["frame_info_len"])),
                       np.transpose(np.matrix(updatedPortScanDf["tcp_options_mss_val"])))

In [11]:
pvalLenMSS
# fcit tests for the null hypothesis that x is independent of y
# low p-value, so given x is independent of y, obtaining the observations from the two
# vectors or more extreme is very unlikely. 
# we reject and may consider that x is not independent of y.
# another consideration for

0.0002389977472441613

In [57]:
dfGraph = updatedPortScanDf.sample(10)

In [64]:
fcit.test(np.transpose(np.matrix(dfGraph["frame_info_len"])),
          np.transpose(np.matrix(dfGraph["tcp_options_mss_val"])))

0.02655792990842002

In [65]:
# there are dependencies from previous iterations - sequential
def eliminateEdges(dfGraph,connectedGraph):
    def reject(node,neighbor,third=None): 
        if third is None:
            third = np.empty([dfGraph[node].shape[0], 0])
        print(node,neighbor)
        pval = fcit.test(np.transpose(np.matrix(dfGraph[node])),
                       np.transpose(np.matrix(dfGraph[neighbor])), 
                        third)
        if pval < 0.05:
            return True
        else:
            return False
    for node in connectedGraph:
        currNeighbors = list(connectedGraph[node])
        while len(currNeighbors) != 0: # neighbor is element in a set
            neighbor = currNeighbors[0]
            if not reject(node,neighbor):
                connectedGraph[node].remove(neighbor)
                connectedGraph[neighbor].remove(node)
            else:
                for third in connectedGraph[neighbor]:
                    if third != node and reject(node,neighbor,
                                                np.transpose(np.matrix(dfGraph[third]))):
                        connectedGraph[node].remove(neighbor)
                        connectedGraph[neighbor].remove(node)
                        break
            currNeighbors = currNeighbors[1:]
    return connectedGraph

In [67]:
#newGraph = eliminateEdges(dfGraph,connectedGraph)
